In [ ]:
import threading
import time

def print_numbers():
  for i in range(10):
    print(f"Number {i}")
    time.sleep(1)

def print_letters():
  for letter in 'abcdefghij':
    print(f"Letter {letter}")
    time.sleep(1.5)

# Create threads
t1 = threading.Thread(target=print_numbers)
t2 = threading.Thread(target=print_letters)

# Start threads
t1.start()
t2.start()

# Wait for both threads to finish
t1.join()
t2.join()

print("Both threads are done!")
